## <font color = yellow> A single Sale will combine the first row from the Buys & Sells dfs

In [ ]:
create_df <- function() {
    
    df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
    df$sale <- as.integer(df$sale)
   #  df$date <- as.Date(df$date, origin = "1899-12-30")
   #  df$true_false <- FALSE  
    return(df)  
    }

# fct call
# df <- create_df() 

In [ ]:
# call fct and create the record keeping df
df <- create_df()
# create empty df
df_buy_rows <- data.frame()

In [ ]:
# create dfs from filtered df
buys <- df[df$type == 'Buy', ]  # Filter buys
sells <- df[df$type == 'Sell', ]  # Filter sells
head(buys, 3)
head(sells,3)

In [ ]:
# A combo fct that creates the dfs used (combo of above )

create_dfs <- function() {
    #fct call
    df <- create_df()   # the first row in this Notebook
    df_buy_rows <- data.frame()
    buys <- df[df$type == 'Buy', ]  # Filter buys
    sells <- df[df$type == 'Sell', ]  # Filter sells
    
    return(list(df = df, df_buy_rows = df_buy_rows, buys = buys, sells = sells))
  }
    

In [ ]:
# Call the combo function, will return a list of dfs
dfs <- create_dfs()

# Access the dataframes from returned list
df <- dfs$df
df_buy_rows <- dfs$df_buy_rows
buys <- dfs$buys
sells <- dfs$sells


    

In [ ]:
# create new dfs that will become the one line dfs (first row ) from buys and sells dfs
s <- data.frame()
b <- data.frame()

# create the Buy & Sell dfs

row_b <- function(b) {
        b <- b
        b <- buys[1,]
        return(b)
    }

    row_s <- function(s) {
        s <- s
        s <- sells[1,]
        return(s)
    }

    

# call fcts


b <- row_b(b)   
s <- row_s(s)    

nrow(b)
nrow(s)

print(b)
print(s)


## <font color = yellow> These values get duplicated b/c no change of i value -  (no loop yet)

In [ ]:
# These values might be default when there is a carryforward ( Buy > Sell )

when_cf <- function (b) {
    
    b <- b
    
    sale <- s$quan
    bs_ratio <- b$quan / s$quan
    cf_quan <- b$quan - s$quan

    b$cf_quan <- cf_quan
    b$cf_pps <- b$pps
    b$cf_basis <- cf_quan * b$pps
    
    b$buy_quan <- s$quan 
    b$buy_pps <-  b$pps
    b$buy_basis <-  b$buy_quan * b$pps

    b$sell_quan <- s$quan
    b$sell_pps <- s$pps
    b$sell_basis <- s$quan * s$pps

    b$profit_loss <- b$sell_basis - b$buy_basis
    b$sale <- sale
    
    s$type <- "SOLD"
    
    return(b)
    
    }
    

In [ ]:
# b <- when_cf(b)
# b

In [ ]:
# modifies the buys df, row 1, with values from a carryforward Sale.

i <- 1   # substitute value acting as a loop value

nrow(buys)

df_records <- function (buys, b) {
    buys <- buys
    b <- b
    
    buys[i, 'cf_quan'] <- b$cf_quan   
    df_buy_rows <<- rbind(df_buy_rows, b)
     
    # buys <- buys[-1, ]  
    
    return(buys)
    
    }
nrow(buys)



In [ ]:
# df_buy_rows <- df_records(buys, b)
# head(df_buy_rows, 3)

In [ ]:
head(buys, 3)

In [ ]:
#  These functions will delete the first row from the Buys and Sells dfs 
#  concluding the first Sale event

i <- 1


# fct
buy_snip <- function (buys) {
    buys <- buys    
    # remove the first row of buys ... is b 
    buys <- buys[-1, ]   
    return(buys)
   }


# fct
sell_snip <- function(sells) {
    sells <- sells    
    sells <- sells[-1, ]    
    return(sells)
    }


# fct call


# nrow(buys)
# nrow(sells)

# buys <- buy_snip(buys)
# sells <- sell_snip(sells)
# nrow(buys)
# nrow(sells)
# head(buys, 3)

In [ ]:
# These are the fcts (in order) to process a Sals, from start to end 

nrow(buys)
nrow(sells)

buys <- buy_snip(buys)
sells <- sell_snip(sells)
nrow(buys)
nrow(sells)
head(buys, 3)

In [ ]:
stopps

## <font color = yellow> The sequence of functions to process a Sale:
    
    dfs <- create_dfs() {
    row_b <- function(b) {
    row_s <- function(s) {
    when_cf <- function (b) {
    df_records <- function (buys, b) 
    buy_snip <- function (buys) {
    sell_snip <- function(sells) {
    

In [ ]:
# fct call: to create vars from the first row of Buy & Sell
b <- row_b(b)   
s <- row_s(s)  
b
s

In [ ]:
# fct call: to populated the fields of B with the Sale data when there is a C/F amound
nrow(buys)
nrow(sells)

# This creates the b df which includes all the fields completed from the Sale of s. This has c/f
b <- when_cf(b) 

b    

In [ ]:
# The Buys df remians unchanged since it was created
head(buys,3)

In [ ]:
# fct call:  First time the fct call creates new df called records from the df b,
#  Subsequent fct calls should append to the df_records for each call or loop

buys <- df_records(buys, b) 

In [ ]:
# The df Buys remains unchanged
head(buys, 3)

In [ ]:
# fct calls: will remove the first row from the buys and sells dfs
nrow(buys)
nrow(sells)
cat("-------------------------------")
buys <- buy_snip(buys)
sells <- sell_snip(sells)

nrow(buys)
nrow(sells)

In [ ]:
# head(buys,3)

head(df_buy_rows)

## <font color = yellow> Works as expected above here. Next, combine fcts into one fct

In [ ]:
# The main body

inc_rows <- function () {
    
    row_b
row_s
when_cf
df_records
buy_snip
sell_snip
    
    buys <- df[df$type == 'Buy', ]  # Filter buys
    sells <- df[df$type == 'Sell', ]  # Filter sells
    print(nrow(buys))
    print(nrow(sells))
 
    buys <- buy_gone(buys, b)
    sells<-sell_gone(sells)
    print(nrow(buys))
    print(nrow(sells))    
    
    }

#  inc_rows()